### Домашнее задание

1. Переписать алгоритм определения центральности через собственные вектора с использованием GPU.
2. Для графа карате клуба вычислить degree centrality, betweenness centrality, eigenvector centrality и pagerank.
3. Сравнить характеристики из п. 2 между собой - насколько они похожи\отличаются и в чем.

## Настройка окружения

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
!pip uninstall cupy-cuda11x --yes
!pip install cupy-cuda12x
!pip install graphviz

## Импорт библиотек

In [2]:
import cupy as cp
import cudf as cd
import cugraph as cg

import warnings
warnings.simplefilter("ignore")

## Вспомогательные функции и классы

In [9]:
class GraphAnalyzer:
    def __init__(self, input_graph:"cugraph.structure.graph_classes.Graph"):
        self.input_graph = input_graph

        self.__all_analyzing_methods = tuple([
           "calc_centrality_by_degree",
           "calc_centrality_by_betweenness",
           "calc_centrality_by_eigenvectors",
           "calc_page_rank"
        ])

    def calc_centrality_by_degree(self) -> cd.DataFrame:
        """
        Рассчитывает и возвращает degree centrality для графа.
        """
        centrality = self.input_graph.degrees()
        return centrality\
               .assign(degree_centrality=centrality["in_degree"] / (len(self.input_graph.nodes()) - 1))\
               .sort_values(by="degree_centrality", ascending=False)

    def calc_centrality_by_betweenness(self) -> cd.DataFrame:
        """
        Рассчитывает и возвращает betweenness centrality для графа.
        """
        centrality = cg.centrality.betweenness_centrality(self.input_graph)
        return centrality.sort_values(by="betweenness_centrality", ascending=False)

    def calc_centrality_by_eigenvectors(self) -> cd.DataFrame:
        """
        Рассчитывает центральность графа через собственные вектора.
        """
        matrix_adjacency = self.input_graph.to_pandas_adjacency()

        ind_vertexes = matrix_adjacency.index
        gpu_matrix = cp.array(matrix_adjacency.values)

        eigenvalues, eigenvectors = cp.linalg.eigh(gpu_matrix)
        ind_max_eigenvalue = cp.argmax(eigenvalues) # Индекс наибольшего собственного значения
        vector_largest_value = eigenvectors[:, ind_max_eigenvalue].flatten().real
        norm = cp.sign(vector_largest_value.sum()) * cp.linalg.norm(vector_largest_value)

        centrality = cd.DataFrame({"vertex": ind_vertexes, "eigen_centrality": vector_largest_value / norm})

        return centrality.sort_values(by="eigen_centrality", ascending=False)

    def calc_page_rank(self) -> cd.DataFrame:
        """
        Рассчитывает pagerank для графа.
        """
        return cg.link_analysis.pagerank(self.input_graph)

    def get_full_analysis(self) -> cd.DataFrame:
        """
        Выполняет все методы для полного анализа графа.
        """
        calculation_result = [getattr(self, name_method)() for name_method in self.__all_analyzing_methods]

        all_stats = calculation_result[0]
        for stat in calculation_result[1:]:
            all_stats = all_stats.merge(stat, on=["vertex"], how="inner")

        return all_stats.sort_values(by=["vertex"])

## Загрузка данных

In [4]:
edges = cd.read_csv("karate-data.csv", sep="\t", names=["source", "destination"])\
        .assign(weights=1)

graph = cg.Graph(directed=True)
graph.from_cudf_edgelist(edges, source="source", destination="destination", edge_attr="weights")

## Анализ графа

In [11]:
analyzer = GraphAnalyzer(input_graph=graph)
analyzer.get_full_analysis()

,vertex,in_degree,out_degree,degree_centrality,betweenness_centrality,eigen_centrality,pagerank
5,1,16,16,0.484848,0.437635,0.355491,0.096999
8,2,9,9,0.272727,0.053937,0.265960,0.052877
7,3,10,10,0.303030,0.143657,0.317193,0.057078
9,4,6,6,0.181818,0.011909,0.211180,0.035860
22,5,3,3,0.090909,0.000631,0.075969,0.021979
14,6,4,4,0.121212,0.029987,0.079483,0.029112
15,7,4,4,0.121212,0.029987,0.079483,0.029112
16,8,4,4,0.121212,0.000000,0.170960,0.024491
11,9,5,5,0.151515,0.055927,0.227404,0.029766
28,10,2,2,0.060606,0.000848,0.102674,0.014309


**Вывод**

На основе таблицы выше видно, что все рассчитаные величины (т. е. degree centrality, betweenness centrality, eigenvector centrality и pagerank) уменьшаются и увеличиваются при уменьшении и увеличении кол-ва связей у вершины соответственно. Отличием является изменчивость величин, которая характеризует то, насколько изменяется каждая величина при изменении степени вершины, что вероятно указывает на разную зависимость величин от кол-ва связей у вершины.


